In [ ]:
import cv2
import numpy as np
import dlib
import time
import pyautogui

In [ ]:
## 인식기 load 및 상수 파라미터 지정
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')
pyautogui.FAILSAFE = False

# 양 눈 영역을 나타내는 랜드마크 인덱스
left_eye = [36, 37, 38, 39, 40, 41]
right_eye = [42, 43, 44, 45, 46, 47]
eyes = [left_eye , right_eye]

threshold_bin = 80 # 이진화 임계값 설정
minimum_size, maximum_size = 5, 1000 # 눈동자 크기 바운더리값
threshold_dis = 100 # 좌표 차이 임계값
sensitivity = 15 # 민감도
output_width, output_height = 1920, 1080 # 출력창 크기 설정
target_x_init, target_y_init = int(output_width/2) , int(output_height/2) # target 좌표 초기값 :: 화면 정중앙
dx = dy = px = py = px_prev = py_prev = 0

In [ ]:
def get_binary_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    # threshold = 200  # 임계값 설정
    _, binary_image = cv2.threshold(blur, threshold_bin, 255, cv2.THRESH_BINARY)

    # 객체 영역과 나머지 영역을 분할하여 결과 이미지 생성
    result = np.where(binary_image <= threshold_bin, 0, 255).astype(np.uint8)

    return result

def crop_and_binarization(img, rect):
    # 이미지 자르기
    x1, y1 = rect[0]
    x2, y2 = rect[2]
    cropped_img = img[y1:y2, x1:x2]

    # 자른 이미지를 이진화
    binary_image = get_binary_img(cropped_img)

    # 객체 영역과 나머지 영역을 분할하여 결과 이미지 생성
    binarized_img = np.where(binary_image <= threshold_bin, 0, 255).astype(np.uint8)

    return binarized_img

In [ ]:
# 이진화된 이미지에 검은색 영역 내 포함될 수 있는 최대크기의 타원을 그리고
# 타원의 중심점을 반환하는 함수
## 조건1: 그릴 수 있는 타원의 사이즈 제한parameters (minimum_size <= 사이즈 <= maximum_size)
## 조건2: 장축/단축의 비율이 2 이상인 너무 길쭉한 타원은 제외
def find_largest_ellipse(binary_img, minimum_size, maximum_size):
    # 0인 픽셀구역의 경계선 검출
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    largest_ellipse = None
    largest_area = 0
    center = None

    for contour in contours:
        # Fit ellipse to contour
        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)

            # Calculate area of the ellipse
            area = np.pi * ellipse[1][0] * ellipse[1][1]
            
            # Check if the area is within the minimum and maximum sizes
            if minimum_size <= area <= maximum_size:
                # Check if the aspect ratio is within the allowed range
                aspect_ratio = ellipse[1][0] / ellipse[1][1]
                if aspect_ratio < 2:
                    # Update the largest ellipse if the current ellipse has a larger area
                    if area > largest_area:
                        largest_ellipse = ellipse
                        largest_area = area

    if(largest_ellipse is not None):
        # Convert ellipse parameters to integers
        center = tuple(map(int, largest_ellipse[0]))
        axes = tuple(map(int, largest_ellipse[1]))
        angle = int(largest_ellipse[2])

        # 이진화 이미지 위에 타원 둘레와,
        cv2.ellipse(binary_img, center, axes, angle, 0, 360, (0, 255, 0), thickness=1)
        # 타원 중심점 그리기
        cv2.circle(binary_img, center, radius=2, color=(0, 255, 0), thickness=-1)

    return center, binary_img

In [ ]:
def init_point():
    pyautogui.moveTo(target_x_init, target_y_init)
    time.sleep(1)

In [ ]:
def get_eye_pts(landmarks, eye):
    # 눈 영역(사각형)의 x, y 좌표를 저장할 리스트 초기화
    eye_pts = []
    # 눈 영역을 crop할 사각 바운더리 계산
    mid_side_h = np.mean([landmarks.part(eye[0]).y, landmarks.part(eye[3]).y])
    up_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[1]).y, landmarks.part(eye[2]).y]))
    up_side = mid_side_h - up_side_dd
    low_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[4]).y, landmarks.part(eye[5]).y]))
    low_side = mid_side_h + low_side_dd
    left_side = landmarks.part(eye[0]).x 
    right_side = landmarks.part(eye[3]).x 

    eye_pts.append((left_side, up_side))
    eye_pts.append((right_side, up_side)) 
    eye_pts.append((right_side, low_side)) 
    eye_pts.append((left_side, low_side))     

    eye_pts_np = np.array(eye_pts, np.int32)

    return eye_pts_np

In [ ]:
def draw_coordinate(frame, coordinate, position):
    if coordinate is not None:
        text = f"({coordinate[0]}, {coordinate[1]})"
    else:
        text = "None"
    
    if position == "left":
        text_position = (10, frame.shape[0] - 10)
    elif position == "right":
        text_position = (frame.shape[1] - 120, frame.shape[0] - 10)
    else:
        raise ValueError("Invalid position. Available positions are 'bottom_left' and 'bottom_right'.")
    
    cv2.putText(frame, text, text_position,
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [ ]:
## main()
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Failed to open webcam")

init_trig = True

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # 영점보정 트리거 시 -> init position of gaze in the screen (center of screen)
    # 첫타만 유효
    if cv2.waitKey(1) & 0xFF == ord('i'):
        init_trig = True
    if init_trig:
        init_point()
        init_trig = False

    # 그레이스케일 후 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        # 얼굴의 68개 landmarks 검출 
        landmarks = predictor(gray, face)
        left_eye_pts_np= get_eye_pts(landmarks, eyes[0])
        right_eye_pts_np= get_eye_pts(landmarks, eyes[1])

    # L, R 안구 이미지 자르고 이진화
    left_eye_biimg = crop_and_binarization(frame, left_eye_pts_np)
    right_eye_biimg = crop_and_binarization(frame, right_eye_pts_np)
    #cv2.imshow("L_eye", left_eye_biimg)
    #cv2.imshow("R_eye", right_eye_biimg)

    # center 반환
    left_center, left_result = find_largest_ellipse(left_eye_biimg, minimum_size, maximum_size)
    right_center, right_result = find_largest_ellipse(right_eye_biimg, minimum_size, maximum_size)
    cv2.imshow("L_eye", left_result)
    cv2.imshow("R_eye", right_result)

    draw_coordinate(frame, left_center, "left")
    draw_coordinate(frame, right_center, "right")

    if left_center is not None and right_center is not None :
        # 좌표는 crop이미지 기준 -> 전체frame이미지 기준 좌표로 transform
        left_new_center = (left_center[0] + left_eye_pts_np[0][0], left_center[1] + left_eye_pts_np[0][1])
        right_new_center = (right_center[0] + right_eye_pts_np[0][0], right_center[1] + right_eye_pts_np[0][1])

        # 이전좌표 차이값과 임계값 비교
        px_prev, py_prev = px, py # 이전 frame의 시선좌표 저장
        px = (left_new_center[0] + right_new_center[0]) // 2 # 현제 frame의 시선좌표
        py = (left_new_center[1] + right_new_center[1]) // 2

        dx, dy = px - px_prev, py - py_prev
        distance = dx**2 + dy**2

        if(distance < threshold_dis):
            dx, dy = 0, 0
            
    current_x, current_y = pyautogui.position()    
    new_x = current_x + (dx * sensitivity)
    new_y = current_y + (dy * sensitivity)
    pyautogui.moveTo(new_x, new_y)  
            
    # 화면에 프레임 출력
    cv2.imshow("Result", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()